In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
rating_data = pd.read_csv("data/test.csv")
#anima_data = pd.read_csv("data/anime.csv")

In [3]:
rating_data = rating_data.drop(columns="Unnamed: 0")

In [4]:
rating_data.shape

(10000000, 6)

In [5]:
count = rating_data['user_id'].value_counts()
count1 = rating_data['anime_id'].value_counts()
rating_data = rating_data[rating_data['user_id'].isin(count[count >= 500].index)].copy()
rating_data = rating_data[rating_data['anime_id'].isin(count1[count1 >= 200].index)].copy()

In [6]:
combine_movie_rating = rating_data.dropna(axis = 0, subset = ['Name'])
movie_ratingCount = (combine_movie_rating.
     groupby(by = ['Name'])['rating'].
     count().
     reset_index()
     [['Name', 'rating']]
    )
movie_ratingCount.head()

,Name,rating
0,"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",561
1,"""Bungaku Shoujo"" Memoire",707
2,"""Bungaku Shoujo"" Movie",1268
3,.hack//G.U. Returner,336
4,.hack//G.U. Trilogy,456


In [7]:
rating_data = combine_movie_rating.merge(movie_ratingCount, left_on = 'Name', right_on = 'Name', how = 'left')
rating_data = rating_data.drop(columns = "rating_x")
rating_data = rating_data.rename(columns={"rating_y": "rating"})
rating_data

,user_id,anime_id,watching_status,watched_episodes,Name,rating
0,85,67,6,0,Basilisk: Kouga Ninpou Chou,1745
1,89,67,6,0,Basilisk: Kouga Ninpou Chou,1745
2,98,67,6,0,Basilisk: Kouga Ninpou Chou,1745
3,108,67,6,0,Basilisk: Kouga Ninpou Chou,1745
4,153,67,2,24,Basilisk: Kouga Ninpou Chou,1745
...,...,...,...,...,...,...
5131529,31942,26359,6,0,Persona 4 the Golden Animation: Thank you Mr. ...,176
5131530,32092,26359,2,1,Persona 4 the Golden Animation: Thank you Mr. ...,176
5131531,32127,26359,6,0,Persona 4 the Golden Animation: Thank you Mr. ...,176
5131532,32140,26359,2,1,Persona 4 the Golden Animation: Thank you Mr. ...,176


In [8]:
# Encoding categorical data
user_ids = rating_data["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
rating_data["user"] = rating_data["user_id"].map(user2user_encoded)
n_users = len(user2user_encoded)

anime_ids = rating_data["anime_id"].unique().tolist()
anime2anime_encoded = {x: i for i, x in enumerate(anime_ids)}
anime_encoded2anime = {i: x for i, x in enumerate(anime_ids)}
rating_data["anime"] = rating_data["anime_id"].map(anime2anime_encoded)
n_animes = len(anime2anime_encoded)

print("Num of users: {}, Num of animes: {}".format(n_users, n_animes))
print("Min rating: {}, Max rating: {}".format(min(rating_data['rating']), max(rating_data['rating'])))

Num of users: 6196, Num of animes: 5540
Min rating: 112, Max rating: 5610


In [9]:
g = rating_data.groupby('user_id')['rating'].count()
top_users = g.dropna().sort_values(ascending=False)[:20]
top_r = rating_data.join(top_users, rsuffix='_r', how='inner', on='user_id')

g = rating_data.groupby('anime_id')['rating'].count()
top_animes = g.dropna().sort_values(ascending=False)[:20]
top_r = top_r.join(top_animes, rsuffix='_r', how='inner', on='anime_id')

pivot = pd.crosstab(top_r.user_id, top_r.anime_id, top_r.rating, aggfunc=np.sum)

In [10]:
pivot.fillna(0, inplace=True)
pivot

anime_id,226,1535,1575,2001,2167,4224,5081,5114,6547,6746,9253,9989,10620,11757,15809,16498,19815,20507,22319,30276
user_id,,,,,,,,,,,,,,,,,,,,
4773,4979.0,5554.0,5302.0,4996.0,4991.0,5525.0,5259.0,5265.0,5543.0,5132.0,5479.0,5012.0,5238.0,5596.0,5076.0,5610.0,5264.0,5140.0,4967.0,5135.0
6536,4979.0,5554.0,5302.0,4996.0,4991.0,5525.0,5259.0,5265.0,5543.0,5132.0,5479.0,5012.0,5238.0,5596.0,5076.0,5610.0,5264.0,5140.0,4967.0,5135.0
10255,4979.0,5554.0,5302.0,4996.0,4991.0,5525.0,5259.0,5265.0,5543.0,5132.0,5479.0,5012.0,5238.0,0.0,5076.0,5610.0,5264.0,5140.0,0.0,5135.0
10367,4979.0,5554.0,5302.0,4996.0,4991.0,5525.0,5259.0,5265.0,5543.0,5132.0,5479.0,5012.0,5238.0,5596.0,5076.0,5610.0,5264.0,5140.0,4967.0,5135.0
10665,4979.0,5554.0,5302.0,4996.0,4991.0,5525.0,5259.0,5265.0,5543.0,5132.0,5479.0,5012.0,5238.0,5596.0,5076.0,5610.0,5264.0,5140.0,4967.0,5135.0
11100,4979.0,5554.0,5302.0,4996.0,4991.0,5525.0,5259.0,5265.0,5543.0,5132.0,5479.0,5012.0,5238.0,5596.0,5076.0,5610.0,5264.0,5140.0,4967.0,5135.0
13423,4979.0,5554.0,5302.0,4996.0,4991.0,5525.0,5259.0,5265.0,5543.0,5132.0,5479.0,5012.0,5238.0,5596.0,5076.0,5610.0,5264.0,5140.0,4967.0,5135.0
14546,4979.0,5554.0,5302.0,4996.0,4991.0,5525.0,5259.0,5265.0,5543.0,5132.0,5479.0,5012.0,5238.0,5596.0,5076.0,5610.0,5264.0,5140.0,4967.0,5135.0
15083,4979.0,5554.0,5302.0,4996.0,0.0,5525.0,5259.0,5265.0,5543.0,5132.0,5479.0,5012.0,5238.0,5596.0,5076.0,5610.0,5264.0,5140.0,4967.0,5135.0


In [11]:
piviot_table = rating_data.pivot_table(index="Name",columns="user_id", values="rating").fillna(0)
piviot_table

user_id,6,12,17,19,21,42,44,47,53,60,...,32564,32566,32570,32575,32578,32582,32587,32589,32595,32600
Name,,,,,,,,,,,,,,,,,,,,,
"""Bungaku Shoujo"" Kyou no Oyatsu: Hatsukoi",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,561.0,0.0,...,0.0,0.0,0.0,561.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Memoire",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,707.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""Bungaku Shoujo"" Movie",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1268.0,1268.0,...,0.0,0.0,0.0,1268.0,0.0,0.0,0.0,0.0,1268.0,0.0
.hack//G.U. Returner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,336.0,0.0,0.0,0.0,0.0
.hack//G.U. Trilogy,0.0,456.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC Kei,0.0,0.0,1464.0,0.0,1464.0,0.0,0.0,0.0,1464.0,0.0,...,1464.0,0.0,1464.0,0.0,0.0,0.0,1464.0,0.0,0.0,0.0
xxxHOLiC Movie: Manatsu no Yoru no Yume,0.0,0.0,954.0,0.0,0.0,0.0,0.0,0.0,954.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,954.0,0.0,0.0,0.0
xxxHOLiC Rou,0.0,0.0,954.0,0.0,0.0,0.0,0.0,0.0,954.0,0.0,...,954.0,0.0,954.0,0.0,0.0,0.0,954.0,0.0,0.0,0.0


In [12]:
from scipy.sparse import csr_matrix
piviot_table_matrix = csr_matrix(piviot_table.values)

In [13]:
piviot_table_matrix

<5539x6196 sparse matrix of type '<class 'numpy.float64'>'
	with 5131310 stored elements in Compressed Sparse Row format>

In [14]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(piviot_table_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [15]:
piviot_table.iloc[2276, :].values.reshape(1, -1)

array([[0., 0., 0., ..., 0., 0., 0.]])

In [16]:
def predict():
    random_anime = np.random.choice(piviot_table.shape[0]) 

    query = piviot_table.iloc[random_anime, :].values.reshape(1, -1)
    distance, suggestions = model.kneighbors(query, n_neighbors=6)

    print(suggestions)
    print(piviot_table.index[random_anime])
    
    for i in range(0, len(distance.flatten())):
        if i == 0:
            print('Recommendations for {0}:\n'.format(piviot_table.index[random_anime]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, piviot_table.index[suggestions.flatten()[i]], distance.flatten()[i]))

In [20]:
predict()

[[3657 3656 3655 3639 3640 3649]]
One Piece: Episode of Nami - Koukaishi no Namida to Nakama no Kizuna
Recommendations for One Piece: Episode of Nami - Koukaishi no Namida to Nakama no Kizuna:

1: One Piece: Episode of Merry - Mou Hitori no Nakama no Monogatari, with distance of 0.28255250663764475:
2: One Piece: Episode of Luffy - Hand Island no Bouken, with distance of 0.2981511431735162:
3: One Piece Film: Strong World Episode 0, with distance of 0.3824382103879008:
4: One Piece Film: Z, with distance of 0.39464926579655324:
5: One Piece Movie 8: Episode of Alabasta - Sabaku no Oujo to Kaizoku-tachi, with distance of 0.4061994007558848:
